In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Reading data
# Train
df_train = pd.read_csv("./data/train.csv")
# Test
# Note that test set has no SalePrice, which is what you are going to predict
# Therefore, you need to get your own "test" set from the train data to evaluate your model
# The test set is evaluated onto kaggle
df_test = pd.read_csv("./data/test.csv")

In [ ]:
print(df_train.info())
print(set(df_train.dtypes.values))

In [ ]:
# Print columns with missing values
print("Columns with nulls:\n",df_train.isna().sum().loc[df_train.isna().sum() > 0])

In [ ]:
# Check if fireplace quality matches fireplaces
df_train.loc[df_train["FireplaceQu"].isna()]["Fireplaces"].describe()

In [ ]:
# Check BsmtFinSF1 and BsmtFinSF2 values when there are no basements
print(df_train.loc[df_train["BsmtFinType1"].isna()]["BsmtFinSF1"].describe())

# Note that FinType2 has some issues
print(df_train.loc[df_train["BsmtFinType2"].isna()]["BsmtFinSF2"].describe())
print(df_train.groupby("BsmtFinType1")["BsmtFinSF1"].describe())
print(df_train.groupby("BsmtFinType2")["BsmtFinSF2"].describe())

# check BsmtExposure
print(df_train.loc[df_train["BsmtFinType1"] == "Unf"][["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF"]]["BsmtExposure"].value_counts())

# check MasVnrType
print(df_train.loc[df_train["MasVnrType"].isna() | df_train["MasVnrArea"].isna()])
print(df_train.groupby("MasVnrType")["MasVnrArea"].describe())
df_train.groupby(["MasVnrType", "Exterior1st", "Exterior2nd", "ExterQual", "ExterCond"])["MasVnrArea"].describe()

# Check electrical
print(df_train["Electrical"].value_counts())

In [ ]:
df_train["LotFrontage"].describe()

In [ ]:
#Fill in missing values
df_train.loc[df_train["Id"] == 949, "BsmtExposure"] = "No"
df_train.loc[(df_train["MasVnrType"].isna())
            & (df_train["MasVnrArea"].isna()), "MasVnrArea"] = 0
df_train.loc[(df_train["BsmtFinType2"].isna())
            & (df_train["BsmtFinSF2"] > 0), "BsmtFinType2"] = "Rec"
df_train.loc[df_train["MasVnrArea"] == 1, "MasVnrArea"] = 0
df_train.loc[df_train["MasVnrArea"] == 288, "MasVnrType"] = "BrkCmn"
df_train.loc[df_train["MasVnrArea"] == 344, "MasVnrType"] = "BrkFace"
df_train.loc[df_train["MasVnrArea"] == 288, "MasVnrType"] = "BrkFace"
df_train.loc[df_train["Electrical"].isna(), "Electrical"] = df_train["Electrical"].value_counts().nlargest(1).index[0]

In [ ]:
# Histogram plot of sale prices
# Note that there are some outliers
df_train["SalePrice"].plot.hist()
print(df_train["SalePrice"].describe())

In [ ]:
# Check 
check_cols = list(df_train.isna().sum()[df_train.isna().sum() != 0].index)
df_train[check_cols].info()

In [ ]:
# Correlation matrix
df_train_numeric = df_train.select_dtypes(exclude="object")
df_train_numeric.corr()
plt.figure(figsize=(16, 9))
sns.heatmap(df_train_numeric.corr().abs())

In [ ]:
# Lot Frontage
sns.histplot(df_train["LotFrontage"], stat="percent", binwidth=10)